In [47]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
#bibliotecas de machine learning
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate, ShuffleSplit

from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn import utils
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,KFold,GroupKFold
import lightgbm as lgb
import gc

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

In [2]:
# Se monta el drive
from google.colab import drive
drive.mount("/content/MyDrive/")

Mounted at /content/MyDrive/


In [11]:
#Importar datos de kaggle
!pip install kaggle # instalación paquete Kaggle
! mkdir ~/.kaggle # Importación base de datos (se crea la dirección)
! cp kaggle.json ~/.kaggle/ # Se accede al archivo .json
! chmod 600 ~/.kaggle/kaggle.json # Se accede al archivo .json
!kaggle datasets download -d mitishaagarwal/patient #Se descarga el .zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
patient.zip: Skipping, found more recently modified local copy (use --force to force download)


In [13]:
!unzip /content/patient.zip # Se descomprime el archivo .zip

Archive:  /content/patient.zip
  inflating: dataset.csv             


In [15]:
pre_datos=pd.read_csv('/content/dataset.csv')
# Se visualizaron los datos, su dimensión y tipo de varible
print(f'Dimensión de los datos: {pre_datos.shape}')
print(pre_datos.dtypes)
pre_datos

Dimensión de los datos: (91713, 85)
encounter_id                     int64
patient_id                       int64
hospital_id                      int64
age                            float64
bmi                            float64
                                ...   
solid_tumor_with_metastasis    float64
apache_3j_bodysystem            object
apache_2_bodysystem             object
Unnamed: 83                    float64
hospital_death                   int64
Length: 85, dtype: object


,encounter_id,patient_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,...,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,Unnamed: 83,hospital_death
0,66154,25312,118,68.0,22.730000,0,Caucasian,M,180.3,Floor,...,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,NaN,0
1,114252,59342,81,77.0,27.420000,0,Caucasian,F,160.0,Floor,...,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,NaN,0
2,119783,50777,118,25.0,31.950000,0,Caucasian,F,172.7,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,NaN,0
3,79267,46918,118,81.0,22.640000,1,Caucasian,F,165.1,Operating Room / Recovery,...,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular,NaN,0
4,92056,34377,33,19.0,NaN,0,Caucasian,M,188.0,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91708,91592,78108,30,75.0,23.060250,0,Caucasian,M,177.8,Floor,...,1.0,0.0,0.0,0.0,0.0,1.0,Sepsis,Cardiovascular,NaN,0
91709,66119,13486,121,56.0,47.179671,0,Caucasian,F,183.0,Floor,...,0.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,NaN,0
91710,8981,58179,195,48.0,27.236914,0,Caucasian,M,170.2,Accident & Emergency,...,1.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,NaN,0
91711,33776,120598,66,NaN,23.297481,0,Caucasian,F,154.9,Accident & Emergency,...,0.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,NaN,0


In [19]:
#Se quitan las columnas con mayor numeros de datos faltantes 
#(Las mayores a 10000)
large_missing = pre_datos.isnull().sum(axis=0).sort_values(ascending=False)[pre_datos.isnull().sum(axis=0).sort_values(ascending=False) > 10000]

#Se deja el dataset con los datos de màs relevancia para diagnostico
pre_datos.drop(large_missing.index.tolist() + ['encounter_id', 'icu_admit_source', 'icu_id', 'icu_stay_type', 'patient_id', 'hospital_id'], 
              axis=1,
             inplace = True)
pre_datos

,age,bmi,elective_surgery,ethnicity,gender,height,icu_type,pre_icu_los_days,weight,apache_2_diagnosis,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
0,68.0,22.730000,0,Caucasian,M,180.3,CTICU,0.541667,73.9,113.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,0
1,77.0,27.420000,0,Caucasian,F,160.0,Med-Surg ICU,0.927778,70.2,108.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0
2,25.0,31.950000,0,Caucasian,F,172.7,Med-Surg ICU,0.000694,95.3,122.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,0
3,81.0,22.640000,1,Caucasian,F,165.1,CTICU,0.000694,61.7,203.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular,0
4,19.0,NaN,0,Caucasian,M,188.0,Med-Surg ICU,0.073611,NaN,119.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Trauma,Trauma,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91708,75.0,23.060250,0,Caucasian,M,177.8,Cardiac ICU,0.298611,72.9,113.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,Sepsis,Cardiovascular,0
91709,56.0,47.179671,0,Caucasian,F,183.0,Med-Surg ICU,0.120139,158.0,113.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,0
91710,48.0,27.236914,0,Caucasian,M,170.2,Med-Surg ICU,0.046528,78.9,123.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,0
91711,NaN,23.297481,0,Caucasian,F,154.9,Med-Surg ICU,0.081944,55.9,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0


In [30]:
#Sustitucion de los datos faltantes por ceros eliminando las filas con dichos datos en bmi,peso,altura

pre_datos= pre_datos[pre_datos[['bmi', 'weight', 'height']].isna().sum(axis=1) == 0]
pre_datos

,age,bmi,elective_surgery,ethnicity,gender,height,icu_type,pre_icu_los_days,weight,apache_2_diagnosis,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
0,68.0,22.730000,0,Caucasian,M,180.3,CTICU,0.541667,73.9,113.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,0
1,77.0,27.420000,0,Caucasian,F,160.0,Med-Surg ICU,0.927778,70.2,108.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0
2,25.0,31.950000,0,Caucasian,F,172.7,Med-Surg ICU,0.000694,95.3,122.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,0
3,81.0,22.640000,1,Caucasian,F,165.1,CTICU,0.000694,61.7,203.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Cardiovascular,Cardiovascular,0
5,67.0,27.560000,0,Caucasian,M,190.5,Med-Surg ICU,0.000694,100.0,301.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91708,75.0,23.060250,0,Caucasian,M,177.8,Cardiac ICU,0.298611,72.9,113.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,Sepsis,Cardiovascular,0
91709,56.0,47.179671,0,Caucasian,F,183.0,Med-Surg ICU,0.120139,158.0,113.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,0
91710,48.0,27.236914,0,Caucasian,M,170.2,Med-Surg ICU,0.046528,78.9,123.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Metabolic,Metabolic,0
91711,NaN,23.297481,0,Caucasian,F,154.9,Med-Surg ICU,0.081944,55.9,108.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0


In [31]:
#Clasificacion de los datos

numerical_cat = [
 'elective_surgery',
 'apache_post_operative',
 'arf_apache',
 'gcs_unable_apache',
 'intubated_apache',
 'ventilated_apache',
 'aids',
 'cirrhosis',
 'diabetes_mellitus',
 'hepatic_failure',
 'immunosuppression',
 'leukemia',
 'lymphoma',
 'solid_tumor_with_metastasis']

categorical = ['ethnicity',
 'gender',
 'icu_type',
 'apache_3j_bodysystem',
 'apache_2_bodysystem']

In [28]:
#Columnas con solo dos elementos distintos (Es una variable binaria)

pre_datos.nunique()[pre_datos.nunique() == 2].index.tolist()

['elective_surgery',
 'gender',
 'apache_post_operative',
 'arf_apache',
 'gcs_unable_apache',
 'intubated_apache',
 'ventilated_apache',
 'aids',
 'cirrhosis',
 'diabetes_mellitus',
 'hepatic_failure',
 'immunosuppression',
 'leukemia',
 'lymphoma',
 'solid_tumor_with_metastasis',
 'hospital_death']

In [34]:
pre_datos.select_dtypes(include='O').columns.values.tolist()

['ethnicity',
 'gender',
 'icu_type',
 'apache_3j_bodysystem',
 'apache_2_bodysystem']

In [36]:
#Mostrar solo los datos numericos

not_numeric = pre_datos[numerical_cat + categorical + ['hospital_death']].columns.tolist()
numeric_only = pre_datos.drop(not_numeric,axis=1).columns.tolist()
numeric_only

['age',
 'bmi',
 'height',
 'pre_icu_los_days',
 'weight',
 'apache_2_diagnosis',
 'apache_3j_diagnosis',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_verbal_apache',
 'heart_rate_apache',
 'map_apache',
 'resprate_apache',
 'temp_apache',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'd1_heartrate_max',
 'd1_heartrate_min',
 'd1_mbp_max',
 'd1_mbp_min',
 'd1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min',
 'd1_resprate_max',
 'd1_resprate_min',
 'd1_spo2_max',
 'd1_spo2_min',
 'd1_sysbp_max',
 'd1_sysbp_min',
 'd1_sysbp_noninvasive_max',
 'd1_sysbp_noninvasive_min',
 'd1_temp_max',
 'd1_temp_min',
 'h1_diasbp_max',
 'h1_diasbp_min',
 'h1_diasbp_noninvasive_max',
 'h1_diasbp_noninvasive_min',
 'h1_heartrate_max',
 'h1_heartrate_min',
 'h1_mbp_max',
 'h1_mbp_min',
 'h1_mbp_noninvasive_max',
 'h1_mbp_noninvasive_min',
 'h1_resprate_max',
 'h1_resprate_min',
 'h1_spo2_max',
 'h1_spo2_min',
 'h1_sysbp_max',
 'h1_sysbp_min',
 'h1_sysbp_n

In [37]:
#Redondear valores numericos categoricos
for col in numerical_cat:
    pre_datos[col] = pre_datos[col].astype('Int64')

#Llenamos valores faltantes por sus respectivas modas
for col in numerical_cat:
    pre_datos[col] = pre_datos[col].fillna(pre_datos[col].mode()[0])

In [38]:
#Listamos la cantidad de valores faltantes en las columnas de tio de dato numerico

pre_datos[numeric_only].isna().sum(axis=0).sort_values(ascending=False)

d1_potassium_min                 9053
d1_potassium_max                 9053
h1_mbp_noninvasive_min           8455
h1_mbp_noninvasive_max           8455
apache_4a_icu_death_prob         7594
apache_4a_hospital_death_prob    7594
h1_diasbp_noninvasive_max        6982
h1_diasbp_noninvasive_min        6982
h1_sysbp_noninvasive_min         6972
h1_sysbp_noninvasive_max         6972
d1_glucose_min                   5458
d1_glucose_max                   5458
h1_mbp_min                       4287
h1_mbp_max                       4287
h1_resprate_min                  4062
h1_resprate_max                  4062
age                              4055
h1_spo2_min                      3925
h1_spo2_max                      3925
temp_apache                      3884
h1_diasbp_min                    3388
h1_diasbp_max                    3388
h1_sysbp_max                     3379
h1_sysbp_min                     3379
h1_heartrate_min                 2621
h1_heartrate_max                 2621
d1_temp_min 

In [51]:
#Columnas con menos de 11000 datos faltantes
split_one = pre_datos[numeric_only].isna().sum(axis=0).sort_values()[pre_datos[numeric_only].isna().sum(axis=0) < 11000].index.tolist()

#columnas con más de 11000 datos faltantes
split_two = pre_datos[numeric_only].isna().sum(axis=0).sort_values()[pre_datos[numeric_only].isna().sum(axis=0) > 11000].index.tolist()

In [52]:
split_one

['bmi',
 'height',
 'pre_icu_los_days',
 'weight',
 'd1_heartrate_min',
 'd1_heartrate_max',
 'd1_sysbp_max',
 'd1_sysbp_min',
 'd1_diasbp_min',
 'd1_diasbp_max',
 'd1_mbp_min',
 'd1_mbp_max',
 'd1_spo2_max',
 'd1_spo2_min',
 'd1_resprate_min',
 'd1_resprate_max',
 'heart_rate_apache',
 'map_apache',
 'd1_sysbp_noninvasive_min',
 'd1_sysbp_noninvasive_max',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'apache_3j_diagnosis',
 'resprate_apache',
 'd1_mbp_noninvasive_max',
 'd1_mbp_noninvasive_min',
 'apache_2_diagnosis',
 'gcs_eyes_apache',
 'gcs_motor_apache',
 'gcs_verbal_apache',
 'd1_temp_min',
 'd1_temp_max',
 'h1_heartrate_min',
 'h1_heartrate_max',
 'h1_sysbp_min',
 'h1_sysbp_max',
 'h1_diasbp_max',
 'h1_diasbp_min',
 'temp_apache',
 'h1_spo2_min',
 'h1_spo2_max',
 'age',
 'h1_resprate_max',
 'h1_resprate_min',
 'h1_mbp_max',
 'h1_mbp_min',
 'd1_glucose_max',
 'd1_glucose_min',
 'h1_sysbp_noninvasive_max',
 'h1_sysbp_noninvasive_min',
 'h1_diasbp_noninvasive_min',


In [43]:
split_two

[]

In [59]:
process_data = pre_datos.dropna(axis=0)

In [60]:
process_data[categorical].nunique()

ethnicity                6
gender                   2
icu_type                 8
apache_3j_bodysystem    11
apache_2_bodysystem     10
dtype: int64

In [61]:
icu_data = pd.get_dummies(process_data,
    prefix='isin',
    prefix_sep='_',
    columns=categorical,
    drop_first=False)
icu_data.reset_index(drop = True, inplace = True)
icu_data

,age,bmi,elective_surgery,height,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,...,isin_Cardiovascular,isin_Gastrointestinal,isin_Haematologic,isin_Metabolic,isin_Neurologic,isin_Renal/Genitourinary,isin_Respiratory,isin_Trauma,isin_Undefined Diagnoses,isin_Undefined diagnoses
0,68.0,22.730000,0,180.3,0.541667,73.9,113.0,502.01,0,0,...,1,0,0,0,0,0,0,0,0,0
1,77.0,27.420000,0,160.0,0.927778,70.2,108.0,203.01,0,0,...,0,0,0,0,0,0,1,0,0,0
2,67.0,27.560000,0,190.5,0.000694,100.0,301.0,403.01,0,0,...,0,0,0,0,1,0,0,0,0,0
3,72.0,28.257052,1,154.9,0.004861,67.8,303.0,1304.08,1,0,...,0,0,0,0,0,0,1,0,0,0
4,46.0,25.845717,0,167.6,0.000000,72.6,108.0,203.01,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56981,47.0,51.439842,1,195.0,0.033333,186.0,304.0,1406.01,1,0,...,0,1,0,0,0,0,0,0,0,0
56982,54.0,19.770448,0,177.8,0.025694,62.5,109.0,108.01,0,0,...,1,0,0,0,0,0,0,0,0,0
56983,75.0,23.060250,0,177.8,0.298611,72.9,113.0,501.06,0,0,...,1,0,0,0,0,0,0,0,0,0
56984,56.0,47.179671,0,183.0,0.120139,158.0,113.0,501.05,0,0,...,1,0,0,0,0,0,0,0,0,0


In [62]:
icu_data.columns = [x.lower() for x in icu_data.columns.tolist()]
icu_data = icu_data.loc[:,~icu_data.columns.duplicated()]

In [63]:
t = icu_data['arf_apache'].dtype
for col in tqdm(icu_data.columns.tolist()):
    if icu_data[col].values.dtype == 'uint8' or t == icu_data[col].values.dtype:
        icu_data[col] = icu_data[col].astype(int)

100%|██████████| 104/104 [00:00<00:00, 1986.76it/s]


In [64]:
icu_data.dtypes

age                              float64
bmi                              float64
elective_surgery                   int64
height                           float64
pre_icu_los_days                 float64
weight                           float64
apache_2_diagnosis               float64
apache_3j_diagnosis              float64
apache_post_operative              int64
arf_apache                         int64
gcs_eyes_apache                  float64
gcs_motor_apache                 float64
gcs_unable_apache                  int64
gcs_verbal_apache                float64
heart_rate_apache                float64
intubated_apache                   int64
map_apache                       float64
resprate_apache                  float64
temp_apache                      float64
ventilated_apache                  int64
d1_diasbp_max                    float64
d1_diasbp_min                    float64
d1_diasbp_noninvasive_max        float64
d1_diasbp_noninvasive_min        float64
d1_heartrate_max